In [8]:
from gensim.models.keyedvectors import KeyedVectors

w2v = KeyedVectors.load_word2vec_format("data/ruwikiruscorpora_0_300_20.bin", binary=True)
w2v.init_sims(True)

In [23]:
#w2v.most_similar(positive=['дом'])
w2v.index2word[:10]

['год_NOUN',
 'весь_DET',
 'человек_NOUN',
 'время_NOUN',
 'становиться_VERB',
 'первый_ADJ',
 'мочь_VERB',
 'также_ADV',
 'город_NOUN',
 'новый_ADJ']

In [34]:
print(w2v.vocab['весь_DET'].index)
w2v.most_similar(positive=['дом_NOUN'])

1


[('особняк_NOUN', 0.7433063387870789),
 ('дома_ADV', 0.724058985710144),
 ('домик_NOUN', 0.7009927034378052),
 ('квартира_NOUN', 0.694819450378418),
 ('усадьба_NOUN', 0.611950695514679),
 ('флигель_NOUN', 0.6017276644706726),
 ('дача_NOUN', 0.6008142828941345),
 ('двор_NOUN', 0.5725634694099426),
 ('комната_NOUN', 0.559021532535553),
 ('дворец_NOUN', 0.5583986043930054)]

In [37]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [50]:
p = morph.parse('стали')[0]
print(p)
print(p.tag)
print(p.normal_form)
print(p.normal_form + '_' + p.tag.POS)

Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.984662, methods_stack=((<DictionaryAnalyzer>, 'стали', 904, 4),))
VERB,perf,intr plur,past,indc
стать
стать_VERB


In [73]:
import re

def norm_word(word):
    p = morph.parse(word)[0]
    if p.tag.POS is None:
        return p.normal_form
    return p.normal_form + '_' + p.tag.POS

def denorm_word(word):
    return word.split('_')[0]

words = w2v.most_similar(positive=[norm_word('квартира'), norm_word('собаки')])
[denorm_word(w[0]) for w in words]

def text_to_norm_words(text):
    text_alpha = re.sub("[^а-яА-Яa-zA-Z]"," ", text)
    words = text_alpha.lower().split()
    words = [norm_word(w) for w in words]
    unknown = [w for w in words if w not in w2v]
    if unknown:
        print(unknown)
    return [w for w in words if w in w2v]

def find_similar(positive, negative):
    similar = w2v.most_similar(
        positive=text_to_norm_words(positive),
        negative=text_to_norm_words(negative)
    )
    return [denorm_word(word[0]) for word in similar]

find_similar('дом', 'красивый')

['красивый_ADJF']


['особняк',
 'дома',
 'домик',
 'квартира',
 'усадьба',
 'флигель',
 'дача',
 'двор',
 'комната',
 'дворец']

In [75]:
[w for w in w2v.index2word if w.startswith('красивый')]

['красивый_ADJ']

In [77]:
set([w.split('_')[1] for w in w2v.index2word])

{'ADJ',
 'ADP',
 'ADV',
 'CCONJ',
 'DET',
 'INTJ',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'VERB'}

In [81]:
len(w2v.word_vec(norm_word('квартира')))

300